In [1]:
import os
import pandas as pd
from tqdm import tqdm

In [2]:
filename = "Raw Data/touchevent.csv"

In [3]:
def decoupling_time(r):
    '''
    This will output in this format: 1970-08-22 19:25:00 
    '''
    s = str(r)
    year, month, day, hour, minute, second = s[:4], s[4:6], s[6:8], s[8:10], s[10:12], s[12:14]
    
    return '-'.join([year, month, day]) + ' ' + ':'.join([hour, minute, second])
#decoupling_time(20201117235750093)

In [4]:
final_data = pd.DataFrame()

row_count = 10000
c = 0
for chunk in tqdm(pd.read_csv(filename, chunksize=row_count)): 
    #deep copy 
    temp = pd.DataFrame(columns=['experimentid', 'userid'])
    temp = chunk[['experimentid', 'userid']]
    
    #turning now time from raw to machine readable ; #every minute has a specific count                                         
    shortened_times = pd.to_datetime(chunk['timestamp'].apply(
        decoupling_time), yearfirst=True).dt.floor('min')
    temp.insert(2, 'timestamp', shortened_times)
    #attenzione: il conteggio potrebbe essere meglio rappresentato
    # da una misura di massimo un numero di tocchi al secondo, oltre 
    #la quale si è outliers o c'è un errore nei dati; quando viene 
    # portato al minuto, la somma può essere divisa per secondi (60)
    
    #grouping 
    df_grouped = temp.groupby(['userid', 'experimentid', 'timestamp']).size()                                         
    df_grouped = df_grouped.reset_index(name='touches')
    
    #appending to a global df
    final_data = pd.concat([final_data, df_grouped], ignore_index=True) 

1584it [00:59, 25.49it/s]/tmp/ipykernel_45258/3962094761.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  shortened_times = pd.to_datetime(chunk['timestamp'].apply(
2337it [01:29, 23.98it/s]/tmp/ipykernel_45258/3962094761.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  shortened_times = pd.to_datetime(chunk['timestamp'].apply(
3318it [02:11, 24.00it/s]/tmp/ipykernel_45258/3962094761.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  shortened_times = pd.to_datetime(chunk['timestamp'].apply(
4116it [02:46, 22.04it/s]/tmp/ipykernel_45258/3962094761.py:11: UserWarni

In [5]:
final_data.to_csv('reduced_file.csv', index=False)
second_grouping = final_data

In [6]:
def decoupling_time2(r):
    '''
    This will output in this format: 1970-08-22 19:25:00 
    '''
    s = str(r)
    year, month, day, hour, minute, second = s[:4], s[5:7], s[8:10], s[-8:-6], s[-5:-3], s[-2:]
    
    return '-'.join([year, month, day]) + ' ' + ':'.join([hour, minute, second])
#decoupling_time('2020-11-27 15:04:00')

In [15]:
# Doing it again but on the whole dataset to avoid duplicates of rows.

filename = 'touch_sensor_Fluve2.csv'
chunk = pd.read_csv(filename)
temp = pd.DataFrame(columns=['experimentid', 'userid'])
temp = chunk[['experimentid', 'userid']]
    
    #turning now time from raw to machine readable ; #every minute has a specific count                                         
shortened_times = pd.to_datetime(chunk['timestamp'].apply(
decoupling_time2), yearfirst=True).dt.floor('min')
temp.insert(2, 'timestamp', shortened_times)
#attenzione: il conteggio potrebbe essere meglio rappresentato
# da una misura di massimo un numero di tocchi al secondo, oltre 
#la quale si è outliers o c'è un errore nei dati; quando viene 
# portato al minuto, la somma può essere divisa per secondi (60)
    
#grouping 
df_grouped = temp.groupby(['userid', 'experimentid', 'timestamp']).size()                                         
df_grouped = df_grouped.reset_index(name='touches')
    
    #appending to a global df


In [16]:
print(df_grouped.shape, "We're done!")  #Now the rows are only 1235536; 
                                        #Run again the cell above to prove that there are no more time duplicates. 

(1235536, 4) We're done!


In [17]:
df_grouped.to_csv('touch_sensor_Fluve2.csv', index=False)